# Library

In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

import time

import requests
import datetime

import os

import pandas as pd

# Parameter

In [39]:
list_base_url = "https://www.yeogi.com/domestic-accommodations?"
region_list = ["경기", "제주도", "충남", "인천", "대구", "대전", "서울", "경남", "부산", "전북", "울산", "광주", "강원", "경북", "전남", "충북", "세종"]
Category_Mapping_table = {
    1 : "Motel",
    2 : "Hotel/Resort",
    5 : "Camping"
}

# DEF : Extract and Transformation

In [ ]:
def weekday_setting_url(base_url, Category, Keyword):
    # cron tab : 0 => 월요일
    monday_date = "2024-11-18" # cron 시 : datetime.datetime.now().strftime("%Y-%m-%d")
    friday_date = "2024-11-22" # cron 시 : (datetime.datetime.now() + datetime.timedelta(days=4)).strftime("%Y-%m-%d")
    
    weekday_parameter = f"keyword={Keyword}&autoKeyword=&checkIn={monday_date}&checkOut={friday_date}&personal=2&freeForm=false&category={Category}&" #page={page}
    
    url = base_url+weekday_parameter
    return url

def holiday_setting_url(base_url, Category, Keyword):
    saturday_date = "2024-11-23" # cron 시 : (datetime.datetime.now() + datetime.timedelta(days=5)).strftime("%Y-%m-%d")
    sunday_date = "2024-11-24" # cron 시 : (datetime.datetime.now() + datetime.timedelta(days=6)).strftime("%Y-%m-%d")
    
    holiday_parameter = f"keyword={Keyword}&autoKeyword=&checkIn={saturday_date}&checkOut={sunday_date}&personal=2&freeForm=false&category={Category}&" #page={page}
    
    url = base_url+holiday_parameter
    return url

#####################
# Setting Dataframe #
#####################
def Make_Basic_DataFrame():
    # Dataframe 셋팅
    accommodations_frame = {
        'ACCOMMODATION_ID': [],
        'ACCOMMODATION_MAINCATEGORY': [],
        'ACCOMMODATION_SUBCATEGORY': [],
        'ACCOMMODATION_NAME': [],
        'LOAD_TIMESTAMP': []
    }
    accommodations_df = pd.DataFrame(accommodations_frame)
        
    price_frame = {
        'ACCOMMODATION_ID': [],
        'DATE_TYPE' : [],
        'PRICE': [],
        'LOAD_TIMESTAMP': []
    }
    price_df = pd.DataFrame(price_frame)
        
    review_frame = {
        'ACCOMMODATION_ID': [],
        'ACCOMMODATION_RATING': [],
        'ACCOMMODATION_REVIEWCOUNT' : [],
        'LOAD_TIMESTAMP': []
    }
    review_df = pd.DataFrame(review_frame)
    return accommodations_df, price_df, review_df

####################
# Crawling Setting #
####################
def crawling_setting():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode (without GUI)
    chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
    chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

    # Initialize the Chrome driver
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
    return driver

##################
# Crawling Basic #
##################
def extract_accommodation_price(element):  
    """Extract price from the accommodation element."""
    try:
        price_element = element.find_element(By.CLASS_NAME, "css-5r5920")
        price_text = price_element.text.strip().replace(',', '')  # Remove commas
        return int(price_text)  # Convert to integer
    except NoSuchElementException:
        print("Price element not found, skipping this element.")
        return None  # Return None if not found
    except Exception as e:
        print(f"An error occurred while extracting the price: {e}")
        return None  # Return None if any other error occurs

def extract_accommodation_data(element):
    """Extract various details from an accommodation element."""
    try:
        # Extract Accommodation ID
        href = element.get_attribute("href")
        Accommodation_ID = href.split('/')[-1].split('?')[0]

        # Extract Name
        Accommodation_Name = element.find_element(By.CLASS_NAME, "gc-thumbnail-type-seller-card-title").text.strip()

        # Extract Rating and Review
        review_rating = element.find_element(By.CLASS_NAME, "css-9ml4lz").text.strip()  # Rating
        review_count_text = element.find_element(By.CLASS_NAME, "css-oj6onp").text.strip()  # Review Count
        review_count_numeric = ''.join(filter(str.isdigit, review_count_text))  # Extract only digits
        review_count = int(review_count_numeric) if review_count_numeric else 0  # Default to 0 if empty

        # Extract Accommodation Subcategory
        ul_element = element.find_element(By.CLASS_NAME, "css-19akvy6")
        li_elements = ul_element.find_elements(By.TAG_NAME, "li")
        Accomodation_SubCategory = "_".join([li.text for li in li_elements])

        return Accommodation_ID, Accommodation_Name, review_rating, review_count, Accomodation_SubCategory
    
    except Exception as e:
        print(f"An error occurred while extracting data: {e}")
        return None  # Return None if any error occurs
    
def update_dataframes(accommodations_df, price_df, review_df, temp_accommodations, temp_prices, temp_reviews):
    """Update DataFrames with new data."""
    accommodations_df = pd.concat([accommodations_df, pd.DataFrame(temp_accommodations)], ignore_index=True)
    price_df = pd.concat([price_df, pd.DataFrame(temp_prices)], ignore_index=True)
    review_df = pd.concat([review_df, pd.DataFrame(temp_reviews)], ignore_index=True)

    print(accommodations_df.head())
    print(price_df.head())
    print(review_df.head())

    return accommodations_df, price_df, review_df

def scrape_accommodation(driver, base_url, date_type, Category, Keyword_list):
    accommodation_df, price_df, review_df = Make_Basic_DataFrame()
    
    for Keyword in Keyword_list:
        print("Crawling Start KeyWord : ", Keyword )
        page = 1
        
        if date_type == "Weekday":
            Keyword_base_url = weekday_setting_url(base_url, Category, Keyword)
        elif date_type == "Holiday":
            Keyword_base_url = holiday_setting_url(base_url, Category, Keyword)   
        else:
            print("Date_Type Error")
            return None, None, None
        
        print("Crawling_baseurl : ", base_url)
        
        # while True:
        while page <= 1:
            url = Keyword_base_url+f"page={page}"
            print("crawling_url : ", url)
            driver.get(url)

            elements = driver.find_elements(By.CLASS_NAME, "gc-thumbnail-type-seller-card")
            
            if not elements:
                print("No more elements found. Exiting...")
                break
            
            temp_accommodations = []
            temp_prices = []
            temp_reviews = []

            current_timestamp = pd.to_datetime('now')
            load_timestamp = current_timestamp.strftime('%Y-%m-%d')
            
            for element in elements:
                # Extract price
                price = extract_accommodation_price(element)
                if price is None:
                    continue  # Skip this iteration if price is not found
                
                # Extract accommodation data
                extracted_data = extract_accommodation_data(element)
                if extracted_data is None:
                    continue  # Skip if there was an error extracting data
                
                Accommodation_ID, Accommodation_Name, review_rating, review_count, Accomodation_SubCategory = extracted_data
            
                temp_accommodations.append({
                        'ACCOMMODATION_ID': Accommodation_ID,
                        'ACCOMMODATION_MAINCATEGORY': Category_Mapping_table.get(Category),
                        'ACCOMMODATION_SUBCATEGORY': Accomodation_SubCategory,
                        'ACCOMMODATION_NAME': Accommodation_Name,
                        'LOAD_TIMESTAMP': load_timestamp
                })

                temp_prices.append({
                    'ACCOMMODATION_ID': Accommodation_ID,
                    'DATE_TYPE': date_type,  # Use the provided date type
                    'PRICE': price,
                    'LOAD_TIMESTAMP': load_timestamp
                })

                temp_reviews.append({
                    'ACCOMMODATION_ID': Accommodation_ID,
                    'ACCOMMODATION_RATING': review_rating,
                    'ACCOMMODATION_REVIEWCOUNT': review_count,
                    'LOAD_TIMESTAMP': load_timestamp
                })

                if len(temp_accommodations) >= 10:
                    break
                
            accommodation_df, price_df, review_df = update_dataframes(accommodation_df, price_df, review_df, temp_accommodations, temp_prices, temp_reviews)
            page += 1
            
    return accommodation_df, price_df, review_df

# DEF : LOAD TO PARQUET

In [42]:
# local -> boto3 s3로 변경 필요
def create_dir_if_not_exists():
    now = datetime.datetime.now().strftime("%Y-%m-%d")
    directory = "./" + now + "_Tables"
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

# Parquet 파일 저장 함수
def create_parquet_Basic(directory, accommodation_df, review_df, price_df):
    # Accommodation 데이터프레임 저장
    accommodation_filename = os.path.join(directory, 'accommodation_table.parquet')
    accommodation_df.to_parquet(accommodation_filename, engine='pyarrow', index=False)
    print(f"저장 완료: {accommodation_filename}")
    
    # Review 데이터프레임 저장
    review_filename = os.path.join(directory, 'review_table.parquet')
    review_df.to_parquet(review_filename, engine='pyarrow', index=False)
    print(f"저장 완료: {review_filename}")
    
    # Price 데이터프레임 저장
    price_filename = os.path.join(directory, 'price_table.parquet')
    price_df.to_parquet(price_filename, engine='pyarrow', index=False)
    print(f"저장 완료: {price_filename}")


# SCRAPING

In [51]:
driver = crawling_setting()
# total_accommodation_df, total_review_df, total_price_df = Make_Basic_DataFrame()
total_accommodation_df, total_price_df, total_review_df = Make_Basic_DataFrame()

for Category in Category_Mapping_table.keys():
    try:
        # Weekday와 Holiday 데이터 수집
        accommodation_Weekday_df, price_Weekday_df, review_Weekday_df = scrape_accommodation(driver, list_base_url, "Weekday", Category, region_list)
        accommodation_Holiday_df, price_Holiday_df, review_Holiday_df = scrape_accommodation(driver, list_base_url, "Holiday", Category, region_list)
        
        # 기본 정보 중복값 제거
        accommodation_df = pd.concat([accommodation_Weekday_df, accommodation_Holiday_df], ignore_index=True)
        accommodation_df = accommodation_df.drop_duplicates(['ACCOMMODATION_ID'])
        accommodation_df = accommodation_df.astype({'ACCOMMODATION_ID': 'int64'})
        total_accommodation_df = pd.concat([total_accommodation_df, accommodation_df], ignore_index=True)

        # 리뷰 데이터 중복값 제거
        review_df = pd.concat([review_Weekday_df, review_Holiday_df], ignore_index=True)
        review_df = review_df.drop_duplicates(['ACCOMMODATION_ID'])
        review_df = review_df.astype({'ACCOMMODATION_ID': 'int64', 'ACCOMMODATION_REVIEWCOUNT': 'int64'})
        total_review_df = pd.concat([total_review_df, review_df], ignore_index=True)

        # 가격 데이터 처리
        price_df = pd.concat([price_Weekday_df, price_Holiday_df], ignore_index=True)
        price_df = price_df.astype({'ACCOMMODATION_ID': 'int64', 'PRICE': 'int64'})
        total_price_df = pd.concat([total_price_df, price_df], ignore_index=True)

    except Exception as e:
        print(f"Error occurred while processing Category {Category}: {e}")
        continue  # 예외 발생 시 다음 Category로 이동

Crawling Start KeyWord :  경기
Crawling_baseurl :  https://www.yeogi.com/domestic-accommodations?
crawling_url :  https://www.yeogi.com/domestic-accommodations?keyword=경기&autoKeyword=&checkIn=2024-11-18&checkOut=2024-11-22&personal=2&freeForm=false&category=1&page=1
  ACCOMMODATION_ID ACCOMMODATION_MAINCATEGORY ACCOMMODATION_SUBCATEGORY  \
0             3468                      Motel                        모텔   
1            46728                      Motel                        모텔   
2              170                      Motel                        모텔   
3            68413                      Motel                        모텔   
4            49868                      Motel                        모텔   

  ACCOMMODATION_NAME LOAD_TIMESTAMP  
0       수원역 J호텔-제이호텔     2024-11-04  
1        의정부 STAGE79     2024-11-04  
2          수원역 하운드호텔     2024-11-04  
3      수원역 HOTEL 테리스     2024-11-04  
4          의정부 케일라호텔     2024-11-04  
  ACCOMMODATION_ID DATE_TYPE    PRICE LOAD_TIMESTAMP
0  

In [10]:
total_accommodation_df

,ACCOMMODATION_ID,ACCOMMODATION_MAINCATEGORY,ACCOMMODATION_SUBCATEGORY,ACCOMMODATION_NAME
0,3468.0,Motel,모텔,수원역 J호텔-제이호텔
1,46728.0,Motel,모텔,의정부 STAGE79
2,170.0,Motel,모텔,수원역 하운드호텔
3,68413.0,Motel,모텔,수원역 HOTEL 테리스
4,49868.0,Motel,모텔,의정부 케일라호텔
...,...,...,...,...
938,70396.0,Camping,캠핑,주노글램핑 논산점 테하스랜드
939,75099.0,Camping,캠핑,홍성 캠프인코지 글램핑 카라반
940,77927.0,Camping,캠핑,강화도 무지개글램핑장
941,71268.0,Camping,캠핑,남해카라반


In [ ]:
total_review_df

In [ ]:
total_price_df

# Load

In [52]:
directory = create_dir_if_not_exists()
create_parquet_Basic(directory, total_accommodation_df, total_review_df, total_price_df)

저장 완료: ./2024-11-04_Tables/accommodation_table.parquet
저장 완료: ./2024-11-04_Tables/review_table.parquet
저장 완료: ./2024-11-04_Tables/price_table.parquet


---

# Crawling Detail

In [25]:
directory = "./2024-11-04_Tables"

accommodation_df = pd.read_parquet(directory+"/accommodation_table.parquet")
# 'ACCOMMODATION_ID'와 'ACCOMMODATION_MAINCATEGORY' 컬럼을 선택합니다.
accommodation_id_category_list = list(zip(accommodation_df['ACCOMMODATION_ID'], accommodation_df['ACCOMMODATION_MAINCATEGORY']))

accommodation_id_category_list

[(3468.0, 'Motel'),
 (46728.0, 'Motel'),
 (170.0, 'Motel'),
 (68413.0, 'Motel'),
 (49868.0, 'Motel'),
 (5203.0, 'Motel'),
 (66620.0, 'Motel'),
 (48986.0, 'Motel'),
 (12546.0, 'Motel'),
 (269.0, 'Motel'),
 (3146.0, 'Motel'),
 (67893.0, 'Motel'),
 (71955.0, 'Motel'),
 (67736.0, 'Motel'),
 (77105.0, 'Motel'),
 (68067.0, 'Motel'),
 (62590.0, 'Motel'),
 (81793.0, 'Motel'),
 (4954.0, 'Motel'),
 (70105.0, 'Motel'),
 (3582.0, 'Motel'),
 (69222.0, 'Motel'),
 (73706.0, 'Motel'),
 (5648.0, 'Motel'),
 (1046.0, 'Motel'),
 (1907.0, 'Motel'),
 (5560.0, 'Motel'),
 (81787.0, 'Motel'),
 (1045.0, 'Motel'),
 (1048.0, 'Motel'),
 (80231.0, 'Motel'),
 (3706.0, 'Motel'),
 (68415.0, 'Motel'),
 (2843.0, 'Motel'),
 (4619.0, 'Motel'),
 (51508.0, 'Motel'),
 (79177.0, 'Motel'),
 (3258.0, 'Motel'),
 (76891.0, 'Motel'),
 (75030.0, 'Motel'),
 (69489.0, 'Motel'),
 (4179.0, 'Motel'),
 (62672.0, 'Motel'),
 (6171.0, 'Motel'),
 (492.0, 'Motel'),
 (1260.0, 'Motel'),
 (4235.0, 'Motel'),
 (80243.0, 'Motel'),
 (62912.0, 'Motel

# DEF : Crawling Detail

In [53]:
def Make_Detail_DataFrame():
    # Dataframe 셋팅
    accommodations_Location_frame = {
        'ACCOMMODATION_ID': [],
        'ACCOMMODATION_LOCATION_MAJOR': [],
        'ACCOMMODATION_LOCATION_MIDDLE': [],
        'ACCOMMODATION_LOCATION_SUB': [],
        'ACCOMMODATION_LOCATION_DETAIL': [],
        'LOAD_TIMESTAMP': []
    }
    Location_df = pd.DataFrame(accommodations_Location_frame)
        
    accommodation_Facilities_frame = {
        'ACCOMMODATION_ID': [],
        'ACCOMMODATION_MAINCATEGORY': [],
        'ACCOMMODATION_FACILITIES': [],
        'LOAD_TIMESTAMP': []
    }
    Facilities_df = pd.DataFrame(accommodation_Facilities_frame)
    
    return Location_df, Facilities_df

def extract_accommodation_location(driver):
    address_span = driver.find_element(By.CSS_SELECTOR, "div.css-z8nsir span.css-1t5t2dt")
    # Get the text from the <span> element
    address_text = address_span.text.strip()

    # Split the text by spaces
    parts = address_text.split()

    # Prepare the variables based on the number of parts
    if len(parts) >= 4:
        first_part = parts[0]
        second_part = parts[1]
        third_part = parts[2]
        fourth_part = ' '.join(parts[3:])  # Join remaining parts as fourth_part
    elif len(parts) == 3:
        first_part = parts[0]
        second_part = parts[1]
        third_part = parts[2]
        fourth_part = None  # Or '' if you prefer an empty string
    else:
        # Handle the case with fewer than 3 parts
        first_part = parts[0] if len(parts) > 0 else None
        second_part = parts[1] if len(parts) > 1 else None
        third_part = parts[2] if len(parts) > 2 else None
        fourth_part = None  # Or however you want to handle this case
    return first_part, second_part, third_part, fourth_part

def extract_accommodation_facilities(driver):
    # 모든 "css-i3rab1" 클래스를 가진 요소를 가져옴
    elements = driver.find_elements(By.CLASS_NAME, "css-i3rab1")

    # 각 요소의 텍스트를 가져와서 리스트에 저장
    texts = [element.text for element in elements]

    # 텍스트를 ','로 구분하여 하나의 문자열로 합침
    Facilities_text = ", ".join(texts)
    return Facilities_text

def update_detail_dataframe(accommodation_id, Location_df, Facilities_df, first_part, second_part, third_part, fourth_part, Facilities, Category, load_timestamp):
    Location_df = pd.concat([Location_df, pd.DataFrame({'ACCOMMODATION_ID': [accommodation_id], 
                                                        'ACCOMMODATION_LOCATION_MAJOR': [first_part], 
                                                        'ACCOMMODATION_LOCATION_MIDDLE': [second_part], 
                                                        'ACCOMMODATION_LOCATION_SUB': [third_part], 
                                                        'ACCOMMODATION_LOCATION_DETAIL': [fourth_part],
                                                        'LOAD_TIMESTAMP': [load_timestamp]})], ignore_index=True)
    
    Facilities_df = pd.concat([Facilities_df, pd.DataFrame({'ACCOMMODATION_ID': [accommodation_id],
                                                            'ACCOMMODATION_MAINCATEGORY' : [Category],
                                                            'ACCOMMODATION_FACILITIES': [Facilities],
                                                            'LOAD_TIMESTAMP': [load_timestamp]})], ignore_index=True)
    
    return Location_df, Facilities_df

def crawl_detail_page(driver, accommodation_id_category_list):
    Location_df, Facilities_df = Make_Detail_DataFrame()
    
    for accommodation_id, main_category in accommodation_id_category_list:
        url = f"https://www.yeogi.com/domestic-accommodations/{accommodation_id}" + "?"
        driver.get(url)
        
        time.sleep(1)
        
        print("crawlingurl : ", url)

        current_timestamp = pd.to_datetime('now')
        load_timestamp = current_timestamp.strftime('%Y-%m-%d')

        # 데이터 추출
        try:
            first_part, second_part, third_part, fourth_part = extract_accommodation_location(driver)
            Facilities = extract_accommodation_facilities(driver)
            
            Location_df, Facilities_df = update_detail_dataframe(
                accommodation_id, Location_df, Facilities_df, 
                first_part, second_part, third_part, fourth_part, 
                Facilities, main_category, load_timestamp
            )
            print("crawlingurl : ", url , "end")
        
        except Exception as e:
            print(f"Error occurred while processing {url}: {e}")
            continue  # 예외 발생 시 다음 요소로 이동
        
        print("end url : ", url)
    
    return Location_df, Facilities_df

# Parquet 파일 저장 함수
def create_parquet_Detail(directory, Location_df, Facilities_df):
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Accommodation 데이터프레임 저장
    Location_filename = os.path.join(directory, 'accommodation_Location_table.parquet')
    Location_df.to_parquet(Location_filename, engine='pyarrow', index=False)
    print(f"저장 완료: {Location_filename}")
    
    # Review 데이터프레임 저장
    Facilities_filename = os.path.join(directory, 'accommodation_Facilities_table.parquet')
    Facilities_df.to_parquet(Facilities_filename, engine='pyarrow', index=False)
    print(f"저장 완료: {Facilities_filename}")

# Testing

In [54]:
driver = crawling_setting()

In [55]:
Location_df, Facilities_df = crawl_detail_page(driver, accommodation_id_category_list)

crawlingurl :  https://www.yeogi.com/domestic-accommodations/3468.0?
crawlingurl :  https://www.yeogi.com/domestic-accommodations/3468.0? end
end url :  https://www.yeogi.com/domestic-accommodations/3468.0?
crawlingurl :  https://www.yeogi.com/domestic-accommodations/46728.0?
crawlingurl :  https://www.yeogi.com/domestic-accommodations/46728.0? end
end url :  https://www.yeogi.com/domestic-accommodations/46728.0?
crawlingurl :  https://www.yeogi.com/domestic-accommodations/170.0?
crawlingurl :  https://www.yeogi.com/domestic-accommodations/170.0? end
end url :  https://www.yeogi.com/domestic-accommodations/170.0?
crawlingurl :  https://www.yeogi.com/domestic-accommodations/68413.0?
crawlingurl :  https://www.yeogi.com/domestic-accommodations/68413.0? end
end url :  https://www.yeogi.com/domestic-accommodations/68413.0?
crawlingurl :  https://www.yeogi.com/domestic-accommodations/49868.0?
crawlingurl :  https://www.yeogi.com/domestic-accommodations/49868.0? end
end url :  https://www.ye

In [32]:
Location_df

,ACCOMMODATION_ID,ACCOMMODATION_LOCATION_MAJOR,ACCOMMODATION_LOCATION_MIDDLE,ACCOMMODATION_LOCATION_SUB,ACCOMMODATION_LOCATION_DETAIL
0,3468,경기,수원시,팔달구,매산로1가 111-4
1,46728,경기,의정부시,의정부동,507-4
2,170,경기,수원시,팔달구,매산로2가 27-11
3,68413,경기,수원시,팔달구,매산로1가 14-15
4,49868,경기,의정부시,의정부동,502-2
...,...,...,...,...,...
506,69334,경북,포항시,남구,구룡포읍 구평리 508-3
507,69137,경북,경주시,구정동,17-1
508,54387,전남,구례군,문척면,중산리 276
509,75250,충북,제천시,봉양읍,구학리 304


In [33]:
Facilities_df

,ACCOMMODATION_ID,ACCOMMODATION_MAINCATEGORY,ACCOMMODATION_FACILITIES
0,3468,Motel,"노래방, 파티룸, 커플 PC, 트윈베드"
1,46728,Motel,"거울룸, 커플 PC, 트윈베드"
2,170,Motel,"파티룸, 트윈베드"
3,68413,Motel,
4,49868,Motel,
...,...,...,...
506,69334,Camping,"샤워실, 바베큐, 드라이기, 무선인터넷, TV, 욕실용품, 에어컨, 냉장고, 객실내..."
507,69137,Camping,"샤워실, 바베큐, 드라이기, 개수대, 무선인터넷, TV, 욕실용품, 매점, 에어컨,..."
508,54387,Camping,"샤워실, 바베큐, 드라이기, TV, 에어컨, 냉장고, 전기밥솥, 금연, 주차장"
509,75250,Camping,"샤워실, 바베큐, 드라이기, 공용샤워실, 욕실용품, 공용화장실, 에어컨, 물놀이시설..."


In [56]:
Location_df = Location_df.astype({'ACCOMMODATION_ID': 'int64'})
Facilities_df = Facilities_df.astype({'ACCOMMODATION_ID': 'int64'})

directory = "./2024-11-03_Tables"
create_parquet_Detail(directory, Location_df, Facilities_df)

저장 완료: ./2024-11-03_Tables/accommodation_Location_table.parquet
저장 완료: ./2024-11-03_Tables/accommodation_Facilities_table.parquet
